# Создание новой базы данных

In [1]:
import pandas as pd
import sqlite3

исправление формата даты в файле клиентопотока

In [2]:
# Загрузка данных из Excel
df = pd.read_excel('excel_files/Clients_traffic.xlsx')

# Разделение даты и времени
split_data = df['Период'].str.split(n=1, expand=True)
split_data.columns = ['date', 'time']

# Замена точек на двоеточия во времени (если время есть)
split_data['time'] = split_data['time'].str.replace('.', ':', regex=False)

# Объединение даты и времени
df['cleaned_date'] = split_data['date'] + ' ' + split_data['time'].fillna('')
df['cleaned_date'] = df['cleaned_date'].str.strip()

# Преобразование в datetime
df['Период'] = pd.to_datetime(df['cleaned_date'], dayfirst=True, errors='coerce')

# Удаление временного столбца (опционально)
df.drop('cleaned_date', axis=1, inplace=True)

df.to_excel("excel_files/Clients_traffic_date_normalized.xlsx", index=False)

In [3]:
def _split_datetime_column(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """Разбиение одного столбца с датой на шесть столбцов"""

    # Создаем копию DataFrame чтобы избежать предупреждений о настройках
    df = df.copy()
    
    # Конвертируем в datetime с обработкой ошибок
    df[col_name] = pd.to_datetime(
        df[col_name],
        format='%d-%m-%Y %H:%M:%S',
        errors='coerce'
    )
    
    # Создаем новые столбцы
    df['год'] = df[col_name].dt.year.astype('Int64')
    df['месяц'] = df[col_name].dt.month.astype('Int64')
    df['день'] = df[col_name].dt.day.astype('Int64')
    df['час'] = df[col_name].dt.hour.astype('Int64')
    df['минута'] = df[col_name].dt.minute.astype('Int64')
    df['секунда'] = df[col_name].dt.second.astype('Int64')
    
    # Удаляем исходный столбец
    return df.drop(columns=[col_name])


def add_weekdays_column(df, column_name, first_day='вторник'):
    """
    Добавляет колонку с днями недели в датафрейм на основе имеющейся колонки с числами.
    
    Параметры:
    - df: входящий датафрейм
    - column_name: название колонки с числами (днями месяца)
    - first_day: день недели, соответствующий первому числу месяца (по умолчанию "Вторник")
    """
    # Множество дней недели
    days_of_week = ['понедельник', 'вторник', 'среда', 'четверг', 'пятница', 'суббота', 'воскресенье']
    
    # Определим индекс дня недели для первого числа месяца
    index_first_day = days_of_week.index(first_day)
    
    # Максимальное количество дней в месяце определяется максимальным числом в колонке
    max_days_in_month = df[column_name].max()
    
    # Создаем полную карту соответствия дней месяца и дней недели
    full_days = []
    for i in range(max_days_in_month):
        full_days.append((index_first_day + i) % 7)
        
    # Формирование словаря {номер дня : день недели}
    mapping_dict = {i+1: days_of_week[d] for i, d in enumerate(full_days)}
    
    # Применение словаря к колонке
    df['день недели'] = df[column_name].apply(lambda x: mapping_dict.get(x))
    
    return df


def _prepare_datetime_files(datetime_files_and_columns: dict):

    for file_path, datetime_column in datetime_files_and_columns.items():
        df = pd.read_excel(f"excel_files/{file_path}")
        datetime_correct_df = _split_datetime_column(df, datetime_column)
        correct_df = add_weekdays_column(datetime_correct_df, 'день')
        # Помечаем, что это скорректированный файл
        unique_df_name = f"{file_path.split('.')[0]}_corrected.xlsx"
        correct_df.to_excel(f"excel_files/{unique_df_name}", index=False)


def create_database(db_name: str, all_file_paths: list, datetime_files_and_columns: dict):
    
    _prepare_datetime_files(datetime_files_and_columns)
    

    # Словарь для хранения данных
    all_data = {}

    # Чтение данных из всех файлов
    for file in all_file_paths:
        data = pd.read_excel(f"excel_files/{file}", sheet_name=None)  # Чтение всех листов
        for sheet_name, df in data.items():
            unique_table_name = f"{file.replace("_corrected", "").split('.')[0]}"  # Уникальное имя таблицы
            all_data[unique_table_name] = df

    # Создание базы данных SQLite
    conn = sqlite3.connect(db_name)

    # Запись данных в базу данных
    for table_name, df in all_data.items():
        print(f"Записываю таблицу: {table_name}")  # Отладочный вывод
        df.to_sql(table_name, conn, if_exists="replace", index=False)

    # Закрытие подключения
    conn.close()

    print("База данных успешно создана")

In [4]:
all_file_paths_example = ["Clients_traffic_date_normalized_corrected.xlsx",
              "Coupons_corrected.xlsx",
              "Coupons_time_distribution.xlsx",
              "Employees_busyness_corrected.xlsx",
              "Employees_productivity_corrected.xlsx",
              "Queues.xlsx"]
datetime_files_and_columns_example = {"Clients_traffic_date_normalized.xlsx": "Период",
                           "Coupons.xlsx": "Время события",
                           "Employees_busyness.xlsx": "Дата",
                           "Employees_productivity.xlsx": "Дата"}

In [5]:
create_database(db_name="database_datetime_corrected.sqlite",
                all_file_paths=all_file_paths_example,
                datetime_files_and_columns=datetime_files_and_columns_example
)

Записываю таблицу: Clients_traffic_date_normalized
Записываю таблицу: Coupons
Записываю таблицу: Coupons_time_distribution
Записываю таблицу: Employees_busyness
Записываю таблицу: Employees_productivity
Записываю таблицу: Queues
База данных успешно создана
